In [1]:
import hddm
import pandas as pd
import kabuki
from IPython.parallel import Client
from radd.hd import stats_df
from myhddm import mle, vis
from myhddm.parse import parse_stats
import statsmodels.api as sm
from scipy import stats
from scipy.stats.stats import pearsonr, spearmanr
import scikits.bootstrap as boot

In [2]:
os.chdir("/home/kyle/Dropbox/PFH/iPFH/HDDM/Final/msm_final")
data=pd.read_csv("/home/kyle/Dropbox/PFH/iPFH/HDDM/Final/allsx_ewma.csv")
idx_med=data.groupby('subj_idx').rt.median().reset_index().rt.values

In [6]:
sxdatalist=[]
for i, (sx, sxdata) in enumerate(data.groupby('subj_idx')):
    sxdata['speed']=np.ones(len(sxdata))
    sxdata.ix[sxdata['rt']<idx_med[i], 'speed']='fast'
    sxdata.ix[sxdata['rt']>=idx_med[i], 'speed']='slow'
    sxdatalist.append(sxdata)
speed_data=pd.concat(sxdatalist)

In [7]:
def run_model(data, jobn):
    
    params=['v','z','t','a','sv', 'sz', 'st']
    m=hddm.HDDM(data, depends_on={'v':['stim','speed']}, 
                informative=False, bias=True, include=params)
    
    m.sample(4000, burn=1500, dbname="speed_simple_traces"+str(jobn)+".db", db='pickle')
    m.print_stats("stats"+str(jobn)+".txt")
    return m

In [8]:
mpath="/home/kyle/Dropbox/PFH/iPFH/HDDM/Final/speed_simple"
os.mkdir(mpath)
os.chdir(mpath)

mlist=[run_model(speed_data, i) for i in range(3)]

 [-----------------100%-----------------] 4001 of 4000 complete in 20874.2 sec

In [9]:
grdf=pd.DataFrame.from_dict(kabuki.analyze.gelman_rubin(mlist), orient='index')
grdf.columns=['gr']
grdf.to_csv("gelman-rubin.csv")

In [10]:
allmodels = kabuki.utils.concat_models(mlist)
allmodels.plot_posteriors(save=True); plt.close('all')
allmodels.print_stats("combined_model_stats.txt")

Plotting a
Plotting a_std
Plotting v(fast.face)
Plotting v(fast.house)
Plotting v(slow.face)
Plotting v(slow.house)
Plotting v_std
Plotting t
Plotting t_std
Plotting sv
Plotting sz
Plotting st
Plotting z
Plotting z_std


In [12]:

allmodels.save('speed_simple')

In [19]:
statsdf=allmodels.gen_stats()
statsdf.reset_index(inplace=True)
statsdf.rename(columns={'index':'param'}, inplace=True)
statsdf.to_csv("combined_model_stats.csv", index=False)